# LangChain4j API Overview

In [1]:
@file:DependsOn("dev.langchain4j:langchain4j:1.0.0-beta1")
@file:DependsOn("dev.langchain4j:langchain4j-open-ai:1.0.0-beta1")

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

In [3]:
import dev.langchain4j.model.openai.OpenAiChatModel
import dev.langchain4j.model.openai.OpenAiChatModelName

val model = OpenAiChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .temperature(0.3)
    .build()

## Chat API

In [4]:
model.chat("How do I create in Kotlin a plus function for a Point data class that adds two Points together in the form point1 + point2?")

In Kotlin, you can overload the `plus` operator for a data class by defining a function named `plus` within the class. For a `Point` data class, you can implement the `plus` function to add the coordinates of two points together. Here's how you can do it:

```kotlin
data class Point(val x: Int, val y: Int) {
    operator fun plus(other: Point): Point {
        return Point(this.x + other.x, this.y + other.y)
    }
}

fun main() {
    val point1 = Point(1, 2)
    val point2 = Point(3, 4)
    
    val result = point1 + point2
    println(result) // Output: Point(x=4, y=6)
}
```

### Explanation:
1. **Data Class Definition**: The `Point` class is defined with two properties, `x` and `y`.
2. **Operator Overloading**: The `operator fun plus(other: Point): Point` function is defined to overload the `+` operator. It takes another `Point` as a parameter and returns a new `Point` whose coordinates are the sum of the coordinates of the two points.
3. **Usage**: In the `main` function, two `Point

## Prompts

### 1 API

In [5]:
import dev.langchain4j.data.message.SystemMessage.systemMessage
import dev.langchain4j.data.message.UserMessage.userMessage
import dev.langchain4j.model.chat.request.ChatRequest
import dev.langchain4j.model.chat.request.chatRequest
import me.kpavlov.langchain4j.kotlin.model.chat.chatAsync

val response = model.chat(
    chatRequest {
        messages += systemMessage(
            """
            You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
        """.trimIndent()
        )
        messages += userMessage(
            """
            What foldRight does? What will be the intermidieate and the end result of the foldRight function call?
            ```kotlin
                "baba".foldRight("") { x, acc -> "${'$'}x${'$'}{acc}${'$'}{acc}" })
            ```
            """.trimIndent()
        )
    }
).aiMessage()
response

AiMessage { text = "The `foldRight` function in Kotlin is a higher-order function that processes elements of a collection (or a string, in this case) from right to left. It takes an initial accumulator value and a lambda function that combines each element with the accumulator.

In your example, you are using `foldRight` on the string `"baba"` with an initial accumulator of an empty string `""`. The lambda function takes two parameters: `x`, which represents the current character being processed, and `acc`, which is the accumulated result from the right side.

Here's a breakdown of how `foldRight` processes the string `"baba"`:

1. Start with the last character: `a`
   - `acc` is `""`
   - Result: `"$x${acc}${acc}"` → `"a"`
   
2. Next character: `b`
   - `acc` is `"a"`
   - Result: `"$x${acc}${acc}"` → `"bab"`

3. Next character: `a`
   - `acc` is `"bab"`
   - Result: `"$x${acc}${acc}"` → `"ababab"`

4. Last character: `b`
   - `acc` is `"ababab"`
   - Result: `"$x${acc}${acc}"` → `"b

### 2 API

In [6]:
interface AssistantFirst {
    fun chat(userMessage: String): String
}

In [7]:
import dev.langchain4j.service.AiServices
import me.kpavlov.langchain4j.kotlin.service.systemMessageProvider

val aiService = AiServices
    .builder(AssistantFirst::class.java)
    .chatLanguageModel(model)
    .systemMessageProvider {
        """
        You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
        """.trimIndent()
    }
    .build()

In [8]:
aiService.chat("""
Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
Make sure to include all imports, but don’t use star imports.
""".trimIndent())

Here's a simple Kotlin program that creates a flow of five elements (1 to 5) and prints each element using the `collect` function. The code follows idiomatic Kotlin conventions and includes all necessary imports without using star imports.

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    createFlow().collect { value ->
        println(value)
    }
}

fun createFlow(): Flow<Int> = flow {
    for (i in 1..5) {
        emit(i)
    }
}
```

### Explanation:
- `runBlocking` is used to start a coroutine that blocks the current thread until its completion.
- `createFlow` function generates a flow of integers from 1 to 5 using the `flow` builder.
- The `collect` function is called to print each emitted value from the flow.

### 3 API

In [9]:
import dev.langchain4j.service.SystemMessage

interface AssistantSecond {

    @SystemMessage(
        """
        You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
        """
    )
    fun chat(userMessage: String): String
}

In [10]:
val assistant = AiServices.create(AssistantSecond::class.java, model)

In [11]:
assistant.chat("""
Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
Make sure to include all imports, but don’t use star imports.
""".trimIndent())

Here's an idiomatic Kotlin code snippet that creates a flow of five elements and prints each one using the `collect` function:

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    createFlow().collect { value ->
        println(value)
    }
}

fun createFlow(): Flow<Int> = flow {
    for (i in 1..5) {
        emit(i)
    }
}
```

### Explanation:
- **Imports**: All necessary imports are included explicitly.
- **Flow Creation**: The `createFlow` function generates a flow of integers from 1 to 5.
- **Collecting Values**: The `collect` function is used to print each emitted value.
- **Coroutine Context**: The `runBlocking` function is used to run the coroutine in a blocking way for demonstration purposes.

## Stream

### Async

In [12]:
import kotlinx.coroutines.runBlocking

runBlocking {
    model.chatAsync {
        messages += systemMessage("You are an expert Kotlin developer.")
        messages += userMessage(
            """
                Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
                Make sure to include all imports, but don’t use star imports.
            """.trimIndent()
        )
    }
}

ChatResponse { aiMessage = AiMessage { text = "Certainly! Below is a complete Kotlin program that creates a flow of five elements (1, 2, 3, 4, and 5) and prints each element using the `collect` function. The necessary imports are included without using star imports.

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.collect

fun main() = runBlocking {
    val numberFlow: Flow<Int> = flow {
        for (i in 1..5) {
            emit(i)
        }
    }

    numberFlow.collect()
}
```

In this code:

- We import the necessary classes from the `kotlinx.coroutines.flow` package.
- We define a `main` function that runs in a coroutine context using `runBlocking`.
- We create a flow of integers from 1 to 5 using the `flow` builder and emit each integer.
- Finally, we collect the flow, which will print each emitted value to the console. 

Make sure you have the appropriate dependencies for Kotli

### Flow

In [13]:
import dev.langchain4j.model.openai.OpenAiStreamingChatModel
import kotlinx.coroutines.flow.Flow
import me.kpavlov.langchain4j.kotlin.model.chat.StreamingChatLanguageModelReply
import me.kpavlov.langchain4j.kotlin.model.chat.chatFlow

val streamModel = OpenAiStreamingChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .temperature(0.3)
    .build()

val flowResponse = streamModel.chatFlow {
    messages += systemMessage("You are an expert Kotlin developer.")
    messages += userMessage(
        """
        Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
        Make sure to include all imports, but don’t use star imports.
        """.trimIndent()
    )
}

runBlocking {
    flowResponse.collect {
        when(it) {
            is StreamingChatLanguageModelReply.CompleteResponse -> print(it.response.aiMessage().text())
            else -> {}
        }
    }
}

Certainly! Below is a Kotlin program that creates a flow of five elements (1, 2, 3, 4, and 5) and prints each element using the `collect` function without any arguments. The necessary imports are included without using star imports.

```kotlin
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.runBlocking

fun main() = runBlocking {
    val numberFlow: Flow<Int> = flow {
        for (i in 1..5) {
            emit(i)
        }
    }

    numberFlow.collect() { value ->
        println(value)
    }
}
```

### Explanation:
1. **Imports**: We import the necessary classes from the `kotlinx.coroutines.flow` package.
2. **Flow Creation**: We create a flow using the `flow` builder, emitting integers from 1 to 5.
3. **Collecting the Flow**: We use the `collect` function to print each emitted value. The `collect` function takes a lambda that receives each emitted value as an argument.
4. **RunBlocking**: The `runBlocking` coroutine builder is used t

### AiService

In [14]:
// Currently under development
//@file:DependsOn("dev.langchain4j:langchain4j-reactor:1.0.0-beta1")
//
//import dev.langchain4j.service.TokenStream
//
//interface AssistantThird {
//    fun chat(message: String): Flow<String>
//}

In [15]:
//val assistant = AiServices.create(AssistantThird::class.java, streamModel)

In [16]:
//val response = assistant.chat("""
//Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
//Make sure to include all imports, but don’t use star imports.
//""".trimIndent())

## Structured Outputs

### JSON_MODE

In [17]:
/**
 * Represents a person with basic personal attributes.
 *
 * @property name The name of the person.
 * @property age The age of the person in years.
 * @property height The height of the person in meters.
 * @property married Indicates if the person is married.
 */
data class Person(val name: String, val age: Int, val height: Double, val married: Boolean)

interface PersonExtractor {
    fun extractPersonFrom(text: String): Person
}

val jsonModel = OpenAiChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .temperature(0.3)
    .responseFormat("json_schema")
    .strictJsonSchema(true)
    .build()

val personExtractor = AiServices.create(PersonExtractor::class.java, jsonModel)

In [18]:
personExtractor.extractPersonFrom(
    """
        John is 42 years old and lives an independent life.
        He stands 1.75 meters tall and carries himself with confidence.
        Currently unmarried, he enjoys the freedom to focus on his personal goals and interests.
    """
)

Person(name=John, age=42, height=1.75, married=false)

In [19]:
data class Step(val explanation: String, val output: String)

data class MathResponse(val steps: List<Step>, val finalAnswer: String)

interface MathAssistant {
    @SystemMessage("You are a helpful math tutor.")
    fun chat(message: String): MathResponse
}

val assistant = AiServices.create(MathAssistant::class.java, jsonModel)


In [20]:
val response = assistant.chat("solve 8x + 31 = 2")

println(response.steps.withIndex().joinToString("\n") { "${it.index + 1}. ${it.value.explanation}" })
println("Answer:")
println(response.finalAnswer)

1. Subtract 31 from both sides of the equation to isolate the term with x.
2. This simplifies to 8x = -29.
3. Now, divide both sides by 8 to solve for x.
4. Simplifying -29 / 8 gives us the final answer.
Answer:
x = -3.625


### Function calling

In [21]:
import dev.langchain4j.agent.tool.Tool

class Tools {
    @Tool
    fun plus(a: Int, b: Int): Int = a + b

    @Tool
    fun minus(a: Int, b: Int): Int = a - b

    @Tool
    fun times(a: Int, b: Int): Int = a * b

    @Tool
    fun div(a: Int, b: Int): Int = a / b
}

interface MathAssistant {
    @SystemMessage("You are a helpful math tutor.")
    fun chat(message: String): String
}

val assistant = AiServices.builder(MathAssistant::class.java)
    .chatLanguageModel(model)
    .tools(Tools())
    .build()

In [22]:
assistant.chat("""
Solve this:
* (5 + 10) * (20 - 3) / 4
* (a + b)^2
* ax^2 + bx + c = 0 for a=1, b=-3, c=-4
*  S = n * (a1 + an) / 2
    n = 10
    a1 = 3
    an = 20
""")

Here are the solutions to each part of your problem:

1. **Calculation of \((5 + 10) * (20 - 3) / 4\)**:
   - \(5 + 10 = 15\)
   - \(20 - 3 = 17\)
   - \((15 * 17) / 4 = 255 / 4 = 42.75\)

2. **Expression \((a + b)^2\)**:
   - This remains as \((a + b)^2\) since \(a\) and \(b\) are variables.

3. **Solving the quadratic equation \(ax^2 + bx + c = 0\) for \(a=1\), \(b=-3\), \(c=-4\)**:
   - The equation becomes \(1x^2 - 3x - 4 = 0\).
   - The solutions are \(x = 1\) and \(x = 4\) (using the quadratic formula).

4. **Sum \(S = n * (a1 + an) / 2\)** with \(n = 10\), \(a1 = 3\), \(an = 20\)**:
   - \(S = 10 * (3 + 20) / 2 = 10 * 23 / 2 = 115\)

So, the final results are:
- The result of the first calculation is **42.75**.
- The expression \((a + b)^2\) remains as is.
- The solutions to the quadratic equation are **1** and **4**.
- The sum \(S\) is **115**.